# Notebook 06: Baselines (XuetangX)

**Purpose:** Implement and evaluate baseline models for next-course prediction.

**Cold-Start Focus:**
- **Global baselines**: Non-personalized models (Popularity, Random)
- **Sequential baselines**: GRU-based models trained on global data
- **Evaluation**: Test on cold-start users (no training data for these users)

**Baselines:**
1. **Random**: Uniform random prediction from vocabulary
2. **Popularity**: Recommend most popular courses from training set
3. **GRU (Global)**: GRU trained on all training pairs, zero-shot evaluation
4. **GRU (Prefix-only)**: GRU using only query prefix (no support set)

**Inputs:**
- `data/processed/xuetangx/episodes/episodes_train_K5_Q10.parquet`
- `data/processed/xuetangx/episodes/episodes_val_K5_Q10.parquet`
- `data/processed/xuetangx/episodes/episodes_test_K5_Q10.parquet`
- `data/processed/xuetangx/pairs/pairs_train.parquet`
- `data/processed/xuetangx/vocab/course2id.json`

**Outputs:**
- Trained models: `models/baselines/random.pkl`, `popularity.pkl`, `gru_global.pth`
- Evaluation results: `results/baselines_K5_Q10.json`
- `reports/06_baselines_xuetangx/<run_tag>/report.json`

**Metrics:**
- Accuracy@1 (exact match)
- Recall@5, Recall@10 (label in top-k)
- MRR (Mean Reciprocal Rank)

**Strategy:**
1. Load episodes and pairs
2. Implement baseline models
3. Train on train episodes
4. Evaluate on val/test episodes
5. Report metrics + save results

In [1]:
# [CELL 06-00] Bootstrap: repo root + paths + logger

import os
import sys
import json
import time
import uuid
import pickle
import hashlib
from pathlib import Path
from datetime import datetime
from typing import Any, Dict, List, Tuple
from collections import Counter

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

t0 = datetime.now()
print(f"[CELL 06-00] start={t0.isoformat(timespec='seconds')}")
print("[CELL 06-00] CWD:", Path.cwd().resolve())

def find_repo_root(start: Path) -> Path:
    start = start.resolve()
    for p in [start, *start.parents]:
        if (p / "PROJECT_STATE.md").exists():
            return p
    raise RuntimeError("Could not find PROJECT_STATE.md. Open notebook from within the repo.")

REPO_ROOT = find_repo_root(Path.cwd())
print("[CELL 06-00] REPO_ROOT:", REPO_ROOT)

PATHS = {
    "META_REGISTRY": REPO_ROOT / "meta.json",
    "DATA_INTERIM": REPO_ROOT / "data" / "interim",
    "DATA_PROCESSED": REPO_ROOT / "data" / "processed",
    "MODELS": REPO_ROOT / "models",
    "RESULTS": REPO_ROOT / "results",
    "REPORTS": REPO_ROOT / "reports",
}
for k, v in PATHS.items():
    print(f"[CELL 06-00] {k}={v}")

def cell_start(cell_id: str, title: str, **kwargs: Any) -> float:
    t = time.time()
    print(f"\n[{cell_id}] {title}")
    print(f"[{cell_id}] start={datetime.now().isoformat(timespec='seconds')}")
    for k, v in kwargs.items():
        print(f"[{cell_id}] {k}={v}")
    return t

def cell_end(cell_id: str, t0: float, **kwargs: Any) -> None:
    for k, v in kwargs.items():
        print(f"[{cell_id}] {k}={v}")
    print(f"[{cell_id}] elapsed={time.time()-t0:.2f}s")
    print(f"[{cell_id}] done")

# Check GPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[CELL 06-00] PyTorch device: {DEVICE}")
print("[CELL 06-00] done")

[CELL 06-00] start=2026-01-07T15:13:46
[CELL 06-00] CWD: C:\anonymous-users-mooc-session-meta\notebooks
[CELL 06-00] REPO_ROOT: C:\anonymous-users-mooc-session-meta
[CELL 06-00] META_REGISTRY=C:\anonymous-users-mooc-session-meta\meta.json
[CELL 06-00] DATA_INTERIM=C:\anonymous-users-mooc-session-meta\data\interim
[CELL 06-00] DATA_PROCESSED=C:\anonymous-users-mooc-session-meta\data\processed
[CELL 06-00] MODELS=C:\anonymous-users-mooc-session-meta\models
[CELL 06-00] RESULTS=C:\anonymous-users-mooc-session-meta\results
[CELL 06-00] REPORTS=C:\anonymous-users-mooc-session-meta\reports
[CELL 06-00] PyTorch device: cuda
[CELL 06-00] done


In [2]:
# [CELL 06-01] Reproducibility: seed everything

t0 = cell_start("CELL 06-01", "Seed everything")

GLOBAL_SEED = 20260107

def seed_everything(seed: int) -> None:
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(GLOBAL_SEED)

cell_end("CELL 06-01", t0, seed=GLOBAL_SEED)


[CELL 06-01] Seed everything
[CELL 06-01] start=2026-01-07T15:13:46
[CELL 06-01] seed=20260107
[CELL 06-01] elapsed=0.00s
[CELL 06-01] done


In [3]:
# [CELL 06-02] JSON/Pickle IO + hashing helpers

t0 = cell_start("CELL 06-02", "IO helpers")

def write_json_atomic(path: Path, obj: Any, indent: int = 2) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    tmp = path.with_suffix(path.suffix + f".tmp_{uuid.uuid4().hex}")
    with tmp.open("w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=indent)
    tmp.replace(path)

def read_json(path: Path) -> Any:
    if not path.exists():
        raise RuntimeError(f"Missing JSON file: {path}")
    with path.open("r", encoding="utf-8") as f:
        return json.load(f)

def save_pickle(path: Path, obj: Any) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("wb") as f:
        pickle.dump(obj, f)

def load_pickle(path: Path) -> Any:
    with path.open("rb") as f:
        return pickle.load(f)

def sha256_file(path: Path, chunk_size: int = 1024 * 1024) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        while True:
            b = f.read(chunk_size)
            if not b:
                break
            h.update(b)
    return h.hexdigest()

cell_end("CELL 06-02", t0)


[CELL 06-02] IO helpers
[CELL 06-02] start=2026-01-07T15:13:46
[CELL 06-02] elapsed=0.00s
[CELL 06-02] done


In [4]:
# [CELL 06-03] Run tagging + config + meta.json

t0 = cell_start("CELL 06-03", "Start run + init files")

NOTEBOOK_NAME = "06_baselines_xuetangx"
RUN_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_ID = uuid.uuid4().hex

OUT_DIR = PATHS["REPORTS"] / NOTEBOOK_NAME / RUN_TAG
OUT_DIR.mkdir(parents=True, exist_ok=True)

REPORT_PATH = OUT_DIR / "report.json"
CONFIG_PATH = OUT_DIR / "config.json"
MANIFEST_PATH = OUT_DIR / "manifest.json"

# Paths
EPISODES_DIR = PATHS["DATA_PROCESSED"] / "xuetangx" / "episodes"
PAIRS_DIR = PATHS["DATA_PROCESSED"] / "xuetangx" / "pairs"
VOCAB_DIR = PATHS["DATA_PROCESSED"] / "xuetangx" / "vocab"
MODELS_DIR = PATHS["MODELS"] / "baselines"
RESULTS_DIR = PATHS["RESULTS"]

MODELS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Focus on K=5, Q=10 for now
K, Q = 5, 10

CFG = {
    "notebook": NOTEBOOK_NAME,
    "run_id": RUN_ID,
    "run_tag": RUN_TAG,
    "seed": GLOBAL_SEED,
    "device": str(DEVICE),
    "k_shot_config": {"K": K, "Q": Q},
    "inputs": {
        "episodes_train": str(EPISODES_DIR / f"episodes_train_K{K}_Q{Q}.parquet"),
        "episodes_val": str(EPISODES_DIR / f"episodes_val_K{K}_Q{Q}.parquet"),
        "episodes_test": str(EPISODES_DIR / f"episodes_test_K{K}_Q{Q}.parquet"),
        "pairs_train": str(PAIRS_DIR / "pairs_train.parquet"),
        "pairs_val": str(PAIRS_DIR / "pairs_val.parquet"),
        "pairs_test": str(PAIRS_DIR / "pairs_test.parquet"),
        "vocab": str(VOCAB_DIR / "course2id.json"),
    },
    "baselines": [
        "random",
        "popularity",
        "gru_global",
        "sasrec",
        "sessionknn",
    ],
    "gru_config": {
        "embedding_dim": 64,
        "hidden_dim": 128,
        "num_layers": 1,
        "dropout": 0.2,
        "batch_size": 256,
        "learning_rate": 0.001,
        "num_epochs": 10,
        "max_seq_len": 50,  # truncate long sequences
    },
    "metrics": ["accuracy@1", "recall@5", "recall@10", "mrr"],
    "outputs": {
        "models_dir": str(MODELS_DIR),
        "results": str(RESULTS_DIR / f"baselines_K{K}_Q{Q}.json"),
        "out_dir": str(OUT_DIR),
    }
}

write_json_atomic(CONFIG_PATH, CFG)

report = {
    "run_id": RUN_ID,
    "notebook": NOTEBOOK_NAME,
    "run_tag": RUN_TAG,
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "repo_root": str(REPO_ROOT),
    "metrics": {},
    "key_findings": [],
    "sanity_samples": {},
    "data_fingerprints": {},
    "notes": [],
}
write_json_atomic(REPORT_PATH, report)

manifest = {"run_id": RUN_ID, "notebook": NOTEBOOK_NAME, "run_tag": RUN_TAG, "artifacts": []}
write_json_atomic(MANIFEST_PATH, manifest)

# meta.json
META_PATH = PATHS["META_REGISTRY"]
if not META_PATH.exists():
    write_json_atomic(META_PATH, {"schema_version": 1, "runs": []})
meta = read_json(META_PATH)
meta["runs"].append({
    "run_id": RUN_ID,
    "notebook": NOTEBOOK_NAME,
    "run_tag": RUN_TAG,
    "out_dir": str(OUT_DIR),
    "created_at": datetime.now().isoformat(timespec="seconds"),
})
write_json_atomic(META_PATH, meta)

print(f"[CELL 06-03] K={K}, Q={Q}")
print(f"[CELL 06-03] Baselines: {CFG['baselines']}")

cell_end("CELL 06-03", t0, out_dir=str(OUT_DIR))


[CELL 06-03] Start run + init files
[CELL 06-03] start=2026-01-07T15:13:46
[CELL 06-03] K=5, Q=10
[CELL 06-03] Baselines: ['random', 'popularity', 'gru_global', 'sasrec', 'sessionknn']
[CELL 06-03] out_dir=C:\anonymous-users-mooc-session-meta\reports\06_baselines_xuetangx\20260107_151346
[CELL 06-03] elapsed=0.01s
[CELL 06-03] done


In [5]:
# [CELL 06-04] Load vocab, episodes, and pairs

t0 = cell_start("CELL 06-04", "Load data")

# Vocab
course2id = read_json(Path(CFG["inputs"]["vocab"]))
id2course = {int(v): k for k, v in course2id.items()}
n_items = len(course2id)
print(f"[CELL 06-04] Vocabulary: {n_items} courses")

# Episodes
episodes_train = pd.read_parquet(CFG["inputs"]["episodes_train"])
episodes_val = pd.read_parquet(CFG["inputs"]["episodes_val"])
episodes_test = pd.read_parquet(CFG["inputs"]["episodes_test"])

print(f"[CELL 06-04] Episodes train: {len(episodes_train):,} episodes")
print(f"[CELL 06-04] Episodes val:   {len(episodes_val):,} episodes")
print(f"[CELL 06-04] Episodes test:  {len(episodes_test):,} episodes")

# Pairs (for GRU training)
pairs_train = pd.read_parquet(CFG["inputs"]["pairs_train"])
pairs_val = pd.read_parquet(CFG["inputs"]["pairs_val"])
pairs_test = pd.read_parquet(CFG["inputs"]["pairs_test"])
print(f"[CELL 06-04] Pairs train: {len(pairs_train):,} pairs")
print(f"[CELL 06-04] Pairs val:   {len(pairs_val):,} pairs")
print(f"[CELL 06-04] Pairs test:  {len(pairs_test):,} pairs")

cell_end("CELL 06-04", t0)


[CELL 06-04] Load data
[CELL 06-04] start=2026-01-07T15:13:47
[CELL 06-04] Vocabulary: 343 courses
[CELL 06-04] Episodes train: 66,187 episodes
[CELL 06-04] Episodes val:   340 episodes
[CELL 06-04] Episodes test:  346 episodes
[CELL 06-04] Pairs train: 212,923 pairs
[CELL 06-04] Pairs val:   24,698 pairs
[CELL 06-04] Pairs test:  26,608 pairs
[CELL 06-04] elapsed=0.23s
[CELL 06-04] done


In [6]:
# [CELL 06-05] Evaluation metrics

t0 = cell_start("CELL 06-05", "Define evaluation metrics")

def compute_metrics(predictions: np.ndarray, labels: np.ndarray, k_values: List[int] = [5, 10]) -> Dict[str, float]:
    """
    Compute ranking metrics.
    
    Args:
        predictions: (n_samples, n_items) score matrix
        labels: (n_samples,) true item indices
        k_values: list of k for Recall@k
    
    Returns:
        dict with accuracy@1, recall@k, mrr
    """
    n_samples = len(labels)
    
    # Get top-k predictions (indices)
    max_k = max(k_values)
    top_k_preds = np.argsort(-predictions, axis=1)[:, :max_k]  # descending order
    
    # Accuracy@1
    top1_preds = top_k_preds[:, 0]
    acc1 = (top1_preds == labels).mean()
    
    # Recall@k
    recall_k = {}
    for k in k_values:
        hits = np.array([labels[i] in top_k_preds[i, :k] for i in range(n_samples)])
        recall_k[f"recall@{k}"] = hits.mean()
    
    # MRR (Mean Reciprocal Rank)
    ranks = []
    for i in range(n_samples):
        # Find rank of true label (1-indexed)
        rank_idx = np.where(top_k_preds[i] == labels[i])[0]
        if len(rank_idx) > 0:
            ranks.append(1.0 / (rank_idx[0] + 1))  # reciprocal rank
        else:
            # Not in top-k, check full ranking
            full_rank = np.where(np.argsort(-predictions[i]) == labels[i])[0][0]
            ranks.append(1.0 / (full_rank + 1))
    mrr = np.mean(ranks)
    
    return {
        "accuracy@1": float(acc1),
        **{k: float(v) for k, v in recall_k.items()},
        "mrr": float(mrr),
    }

print("[CELL 06-05] Metrics: accuracy@1, recall@5, recall@10, mrr")

cell_end("CELL 06-05", t0)


[CELL 06-05] Define evaluation metrics
[CELL 06-05] start=2026-01-07T15:13:47
[CELL 06-05] Metrics: accuracy@1, recall@5, recall@10, mrr
[CELL 06-05] elapsed=0.00s
[CELL 06-05] done


In [7]:
# [CELL 06-06] Baseline 1: Random predictor

t0 = cell_start("CELL 06-06", "Random baseline")

class RandomPredictor:
    def __init__(self, n_items: int, seed: int = 42):
        self.n_items = n_items
        self.rng = np.random.RandomState(seed)
    
    def predict(self, n_samples: int) -> np.ndarray:
        """Return uniform random scores for each item."""
        return self.rng.rand(n_samples, self.n_items)

# Initialize
random_model = RandomPredictor(n_items=n_items, seed=GLOBAL_SEED)

# Save
save_pickle(MODELS_DIR / "random.pkl", random_model)
print(f"[CELL 06-06] Saved: {MODELS_DIR / 'random.pkl'}")

cell_end("CELL 06-06", t0)


[CELL 06-06] Random baseline
[CELL 06-06] start=2026-01-07T15:13:47
[CELL 06-06] Saved: C:\anonymous-users-mooc-session-meta\models\baselines\random.pkl
[CELL 06-06] elapsed=0.00s
[CELL 06-06] done


In [8]:
# [CELL 06-07] Baseline 2: Popularity predictor

t0 = cell_start("CELL 06-07", "Popularity baseline")

class PopularityPredictor:
    def __init__(self, n_items: int):
        self.n_items = n_items
        self.popularity_scores = np.zeros(n_items)
    
    def fit(self, labels: List[int]):
        """Compute popularity from training labels."""
        counts = Counter(labels)
        for item_id, count in counts.items():
            self.popularity_scores[item_id] = count
        # Normalize
        self.popularity_scores /= self.popularity_scores.sum()
    
    def predict(self, n_samples: int) -> np.ndarray:
        """Return popularity scores repeated for each sample."""
        return np.tile(self.popularity_scores, (n_samples, 1))

# Train: extract labels from training pairs
train_labels = pairs_train["label"].tolist()

popularity_model = PopularityPredictor(n_items=n_items)
popularity_model.fit(train_labels)

# Top-5 most popular courses
top5_idx = np.argsort(-popularity_model.popularity_scores)[:5]
print(f"[CELL 06-07] Top-5 popular courses:")
for rank, idx in enumerate(top5_idx, 1):
    course = id2course[idx]
    score = popularity_model.popularity_scores[idx]
    print(f"  {rank}. {course} (score={score:.4f})")

# Save
save_pickle(MODELS_DIR / "popularity.pkl", popularity_model)
print(f"\n[CELL 06-07] Saved: {MODELS_DIR / 'popularity.pkl'}")

cell_end("CELL 06-07", t0)


[CELL 06-07] Popularity baseline
[CELL 06-07] start=2026-01-07T15:13:47
[CELL 06-07] Top-5 popular courses:
  1. course-v1:TsinghuaX+30640014+2015_T2 (score=0.0422)
  2. course-v1:TsinghuaX+30640014+sp (score=0.0349)
  3. TsinghuaX/80511503X/2015_T2 (score=0.0236)
  4. course-v1:TsinghuaX+80512073X_2015_2+2015_T2 (score=0.0233)
  5. course-v1:TsinghuaX+30240184+2015_T2 (score=0.0221)

[CELL 06-07] Saved: C:\anonymous-users-mooc-session-meta\models\baselines\popularity.pkl
[CELL 06-07] elapsed=0.02s
[CELL 06-07] done


In [9]:
# [CELL 06-08] Baseline 3: GRU model definition

t0 = cell_start("CELL 06-08", "Define GRU model")

class GRURecommender(nn.Module):
    def __init__(self, n_items: int, embedding_dim: int, hidden_dim: int, num_layers: int, dropout: float):
        super().__init__()
        self.n_items = n_items
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(n_items, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_dim, n_items)
    
    def forward(self, seq: torch.Tensor, lengths: torch.Tensor) -> torch.Tensor:
        """
        Args:
            seq: (batch, max_len) padded sequences
            lengths: (batch,) actual lengths
        Returns:
            logits: (batch, n_items)
        """
        # Embed
        emb = self.embedding(seq)  # (batch, max_len, embed_dim)
        
        # Pack for efficiency
        packed = nn.utils.rnn.pack_padded_sequence(emb, lengths.cpu(), batch_first=True, enforce_sorted=False)
        
        # GRU
        _, hidden = self.gru(packed)  # hidden: (num_layers, batch, hidden_dim)
        
        # Use last layer hidden state
        h = hidden[-1]  # (batch, hidden_dim)
        
        # Predict
        logits = self.fc(h)  # (batch, n_items)
        return logits

print("[CELL 06-08] GRU model defined")
print(f"  - Embedding dim: {CFG['gru_config']['embedding_dim']}")
print(f"  - Hidden dim: {CFG['gru_config']['hidden_dim']}")
print(f"  - Num layers: {CFG['gru_config']['num_layers']}")

cell_end("CELL 06-08", t0)


[CELL 06-08] Define GRU model
[CELL 06-08] start=2026-01-07T15:13:47
[CELL 06-08] GRU model defined
  - Embedding dim: 64
  - Hidden dim: 128
  - Num layers: 1
[CELL 06-08] elapsed=0.00s
[CELL 06-08] done


In [10]:
# [CELL 06-08B] Baseline 4: SASRec model definition

t0 = cell_start("CELL 06-08B", "Define SASRec model")

class SASRec(nn.Module):
    def __init__(self, n_items: int, hidden_dim: int, num_heads: int, num_blocks: int, max_len: int, dropout: float):
        super().__init__()
        self.n_items = n_items
        self.hidden_dim = hidden_dim
        self.max_len = max_len
        
        # Item embedding + positional embedding
        self.item_emb = nn.Embedding(n_items + 1, hidden_dim, padding_idx=0)  # +1 for padding
        self.pos_emb = nn.Embedding(max_len, hidden_dim)
        self.dropout = nn.Dropout(dropout)
        
        # Multi-head self-attention blocks
        self.blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=hidden_dim,
                nhead=num_heads,
                dim_feedforward=hidden_dim * 4,
                dropout=dropout,
                activation='relu',
                batch_first=True,
            )
            for _ in range(num_blocks)
        ])
        
        self.layer_norm = nn.LayerNorm(hidden_dim)
        
        # Prediction head
        self.fc = nn.Linear(hidden_dim, n_items)
    
    def forward(self, seq: torch.Tensor) -> torch.Tensor:
        """
        Args:
            seq: (batch, seq_len) padded sequences
        Returns:
            logits: (batch, n_items)
        """
        batch_size, seq_len = seq.size()
        
        # Create position indices
        positions = torch.arange(seq_len, device=seq.device).unsqueeze(0).expand(batch_size, -1)
        
        # Embed items and positions
        item_emb = self.item_emb(seq)  # (batch, seq_len, hidden_dim)
        pos_emb = self.pos_emb(positions)  # (batch, seq_len, hidden_dim)
        x = self.dropout(item_emb + pos_emb)
        
        # Create attention mask (causal mask: can only attend to past)
        attn_mask = torch.triu(torch.ones(seq_len, seq_len, device=seq.device) * float('-inf'), diagonal=1)
        
        # Apply transformer blocks
        for block in self.blocks:
            x = block(x, src_mask=attn_mask)
        
        x = self.layer_norm(x)
        
        # Use last position for prediction
        x = x[:, -1, :]  # (batch, hidden_dim)
        
        logits = self.fc(x)  # (batch, n_items)
        return logits

print("[CELL 06-08B] SASRec model defined")
print(f"  - Hidden dim: 64")
print(f"  - Num heads: 2")
print(f"  - Num blocks: 2")
print(f"  - Max len: 50")

cell_end("CELL 06-08B", t0)


[CELL 06-08B] Define SASRec model
[CELL 06-08B] start=2026-01-07T15:13:47
[CELL 06-08B] SASRec model defined
  - Hidden dim: 64
  - Num heads: 2
  - Num blocks: 2
  - Max len: 50
[CELL 06-08B] elapsed=0.00s
[CELL 06-08B] done


In [11]:
# [CELL 06-09] GRU dataset and dataloader

t0 = cell_start("CELL 06-09", "Create GRU dataset")

class PairDataset(Dataset):
    def __init__(self, pairs_df: pd.DataFrame, max_seq_len: int):
        self.pairs = pairs_df.reset_index(drop=True)
        self.max_seq_len = max_seq_len
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        row = self.pairs.iloc[idx]
        prefix = row["prefix"]  # list of item IDs
        label = row["label"]    # int
        
        # Truncate if too long
        if len(prefix) > self.max_seq_len:
            prefix = prefix[-self.max_seq_len:]
        
        return {
            "prefix": prefix,
            "label": label,
            "length": len(prefix),
        }

def collate_fn(batch):
    """Collate batch with padding."""
    prefixes = [item["prefix"] for item in batch]
    labels = [item["label"] for item in batch]
    lengths = [item["length"] for item in batch]
    
    # Pad sequences
    max_len = max(lengths)
    padded = []
    for seq in prefixes:
        padded.append(list(seq) + [0] * (max_len - len(seq)))  # convert to list if numpy array
    
    return {
        "prefix": torch.LongTensor(padded),
        "label": torch.LongTensor(labels),
        "length": torch.LongTensor(lengths),
    }

# Create datasets
train_dataset = PairDataset(pairs_train, max_seq_len=CFG["gru_config"]["max_seq_len"])
train_loader = DataLoader(
    train_dataset, 
    batch_size=CFG["gru_config"]["batch_size"], 
    shuffle=True, 
    collate_fn=collate_fn,
    num_workers=0,  # Windows compatibility
)

print(f"[CELL 06-09] Train dataset: {len(train_dataset):,} pairs")
print(f"[CELL 06-09] Train loader: {len(train_loader):,} batches")

cell_end("CELL 06-09", t0)


[CELL 06-09] Create GRU dataset
[CELL 06-09] start=2026-01-07T15:13:47
[CELL 06-09] Train dataset: 212,923 pairs
[CELL 06-09] Train loader: 832 batches
[CELL 06-09] elapsed=0.01s
[CELL 06-09] done


In [12]:
# [CELL 06-10] Train GRU model

t0 = cell_start("CELL 06-10", "Train GRU")

# Initialize model
gru_model = GRURecommender(
    n_items=n_items,
    embedding_dim=CFG["gru_config"]["embedding_dim"],
    hidden_dim=CFG["gru_config"]["hidden_dim"],
    num_layers=CFG["gru_config"]["num_layers"],
    dropout=CFG["gru_config"]["dropout"],
).to(DEVICE)

optimizer = torch.optim.Adam(gru_model.parameters(), lr=CFG["gru_config"]["learning_rate"])
criterion = nn.CrossEntropyLoss()

print(f"[CELL 06-10] Model parameters: {sum(p.numel() for p in gru_model.parameters()):,}")

# Training loop
gru_model.train()
for epoch in range(CFG["gru_config"]["num_epochs"]):
    epoch_loss = 0.0
    for batch in train_loader:
        prefix = batch["prefix"].to(DEVICE)
        label = batch["label"].to(DEVICE)
        length = batch["length"].to(DEVICE)
        
        optimizer.zero_grad()
        logits = gru_model(prefix, length)
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    print(f"[CELL 06-10] Epoch {epoch+1}/{CFG['gru_config']['num_epochs']}: loss={avg_loss:.4f}")

# Save model
torch.save(gru_model.state_dict(), MODELS_DIR / "gru_global.pth")
print(f"\n[CELL 06-10] Saved: {MODELS_DIR / 'gru_global.pth'}")

cell_end("CELL 06-10", t0)


[CELL 06-10] Train GRU
[CELL 06-10] start=2026-01-07T15:13:47
[CELL 06-10] Model parameters: 140,695
[CELL 06-10] Epoch 1/10: loss=3.8468
[CELL 06-10] Epoch 2/10: loss=3.2844
[CELL 06-10] Epoch 3/10: loss=3.1635
[CELL 06-10] Epoch 4/10: loss=3.0967
[CELL 06-10] Epoch 5/10: loss=3.0510
[CELL 06-10] Epoch 6/10: loss=3.0146
[CELL 06-10] Epoch 7/10: loss=2.9851
[CELL 06-10] Epoch 8/10: loss=2.9586
[CELL 06-10] Epoch 9/10: loss=2.9354
[CELL 06-10] Epoch 10/10: loss=2.9141

[CELL 06-10] Saved: C:\anonymous-users-mooc-session-meta\models\baselines\gru_global.pth
[CELL 06-10] elapsed=198.39s
[CELL 06-10] done


In [13]:
# [CELL 06-10B] Train SASRec model

t0 = cell_start("CELL 06-10B", "Train SASRec")

# SASRec config
sasrec_config = {
    'hidden_dim': 64,
    'num_heads': 2,
    'num_blocks': 2,
    'max_len': 50,
    'dropout': 0.2,
    'batch_size': 256,
    'learning_rate': 0.001,
    'num_epochs': 10,
}

# Initialize model
sasrec_model = SASRec(
    n_items=n_items,
    hidden_dim=sasrec_config['hidden_dim'],
    num_heads=sasrec_config['num_heads'],
    num_blocks=sasrec_config['num_blocks'],
    max_len=sasrec_config['max_len'],
    dropout=sasrec_config['dropout'],
).to(DEVICE)

optimizer = torch.optim.Adam(sasrec_model.parameters(), lr=sasrec_config['learning_rate'])
criterion = nn.CrossEntropyLoss()

print(f"[CELL 06-10B] SASRec parameters: {sum(p.numel() for p in sasrec_model.parameters()):,}")

# Create SASRec dataloader (same as GRU)
def collate_fn_sasrec(batch):
    prefixes = [item['prefix'] for item in batch]
    labels = [item['label'] for item in batch]
    
    # Pad to same length
    max_len = max(len(p) for p in prefixes)
    padded = []
    for seq in prefixes:
        if len(seq) > sasrec_config['max_len']:
            seq = seq[-sasrec_config['max_len']:]
        padded.append(list(seq) + [0] * (max_len - len(seq)))
    
    return {
        'prefix': torch.LongTensor(padded),
        'label': torch.LongTensor(labels),
    }

sasrec_loader = DataLoader(
    train_dataset,
    batch_size=sasrec_config['batch_size'],
    shuffle=True,
    collate_fn=collate_fn_sasrec,
    num_workers=0,
)

# Training loop
sasrec_model.train()
for epoch in range(sasrec_config['num_epochs']):
    epoch_loss = 0.0
    for batch in sasrec_loader:
        prefix = batch['prefix'].to(DEVICE)
        label = batch['label'].to(DEVICE)
        
        optimizer.zero_grad()
        logits = sasrec_model(prefix)
        loss = criterion(logits, label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(sasrec_loader)
    print(f"[CELL 06-10B] Epoch {epoch+1}/{sasrec_config['num_epochs']}: loss={avg_loss:.4f}")

# Save
torch.save(sasrec_model.state_dict(), MODELS_DIR / 'sasrec.pth')
print(f"\n[CELL 06-10B] Saved: {MODELS_DIR / 'sasrec.pth'}")

cell_end("CELL 06-10B", t0)


[CELL 06-10B] Train SASRec
[CELL 06-10B] start=2026-01-07T15:17:05
[CELL 06-10B] SASRec parameters: 147,607
[CELL 06-10B] Epoch 1/10: loss=4.3654
[CELL 06-10B] Epoch 2/10: loss=3.9053
[CELL 06-10B] Epoch 3/10: loss=3.7789
[CELL 06-10B] Epoch 4/10: loss=3.7058
[CELL 06-10B] Epoch 5/10: loss=3.6541
[CELL 06-10B] Epoch 6/10: loss=3.6153
[CELL 06-10B] Epoch 7/10: loss=3.5836
[CELL 06-10B] Epoch 8/10: loss=3.5589
[CELL 06-10B] Epoch 9/10: loss=3.5389
[CELL 06-10B] Epoch 10/10: loss=3.5191

[CELL 06-10B] Saved: C:\anonymous-users-mooc-session-meta\models\baselines\sasrec.pth
[CELL 06-10B] elapsed=294.43s
[CELL 06-10B] done


In [14]:
# [CELL 06-10C] Baseline 5: Session-KNN (V-SKNN)

t0 = cell_start("CELL 06-10C", "Session-KNN baseline")

class SessionKNN:
    """
    Session-based k-Nearest Neighbors (V-SKNN variant).

    Finds similar past sessions and recommends items from those sessions.
    Similarity based on cosine similarity of session item sets.
    """
    def __init__(self, n_items: int, k: int = 100, sample_size: int = 500):
        self.n_items = n_items
        self.k = k  # number of nearest neighbor sessions
        self.sample_size = sample_size  # max sessions to consider (for efficiency)
        self.sessions = []  # list of (session_id, item_list)

    def fit(self, pairs_df: pd.DataFrame):
        """Build session database from training data."""
        # Group pairs by user to create sessions
        sessions_list = []
        for user_id, user_pairs in pairs_df.groupby('user_id'):
            # Sort by timestamp
            user_pairs_sorted = user_pairs.sort_values('label_ts_epoch')

            # Extract session as list of items (prefix + label)
            session_items = []
            for _, row in user_pairs_sorted.iterrows():
                prefix = row['prefix']
                label = row['label']
                # Add items from prefix
                if isinstance(prefix, (list, np.ndarray)):
                    session_items.extend(prefix)
                # Add label
                session_items.append(label)

            # Store unique items in session
            session_items_unique = list(dict.fromkeys(session_items))  # preserve order, remove duplicates
            sessions_list.append((user_id, session_items_unique))

        self.sessions = sessions_list
        print(f"[CELL 06-10C] Built session database: {len(self.sessions):,} sessions")

    def _session_similarity(self, session1: List[int], session2: List[int]) -> float:
        """Compute cosine similarity between two sessions (item sets)."""
        set1 = set(session1)
        set2 = set(session2)

        if len(set1) == 0 or len(set2) == 0:
            return 0.0

        # Jaccard similarity (intersection over union) as approximation
        # More efficient than full cosine for sets
        intersection = len(set1 & set2)
        union = len(set1 | set2)

        return intersection / union if union > 0 else 0.0

    def predict(self, current_session: List[int]) -> np.ndarray:
        """
        Predict item scores based on k most similar past sessions.

        Args:
            current_session: List of items in current session (prefix)

        Returns:
            scores: (n_items,) array of prediction scores
        """
        if len(current_session) == 0 or len(self.sessions) == 0:
            return np.zeros(self.n_items)

        # Sample sessions for efficiency if too many
        sessions_to_consider = self.sessions
        if len(self.sessions) > self.sample_size:
            import random
            sessions_to_consider = random.sample(self.sessions, self.sample_size)

        # Compute similarities to all sessions
        similarities = []
        for session_id, session_items in sessions_to_consider:
            sim = self._session_similarity(current_session, session_items)
            similarities.append((session_id, session_items, sim))

        # Sort by similarity (descending)
        similarities.sort(key=lambda x: x[2], reverse=True)

        # Take top-k similar sessions
        top_k_sessions = similarities[:self.k]

        # Aggregate scores from top-k sessions
        scores = np.zeros(self.n_items)
        total_sim = sum(sim for _, _, sim in top_k_sessions)

        if total_sim == 0:
            return scores

        for session_id, session_items, sim in top_k_sessions:
            # Weight by similarity and recency (last items have higher weight)
            for i, item in enumerate(session_items):
                if 0 <= item < self.n_items:
                    # Recency weight: more recent items (later in list) get higher weight
                    recency_weight = (i + 1) / len(session_items)
                    scores[item] += sim * recency_weight

        # Normalize by total similarity
        scores /= total_sim

        return scores

# Train Session-KNN
print("[CELL 06-10C] Training Session-KNN (k=100)...")
sessionknn_model = SessionKNN(n_items=n_items, k=100, sample_size=500)
sessionknn_model.fit(pairs_train)

# Save
save_pickle(MODELS_DIR / 'sessionknn.pkl', sessionknn_model)
print(f"[CELL 06-10C] Saved: {MODELS_DIR / 'sessionknn.pkl'}")

cell_end("CELL 06-10C", t0)


[CELL 06-10C] Session-KNN baseline
[CELL 06-10C] start=2026-01-07T15:22:00
[CELL 06-10C] Training Session-KNN (k=100)...
[CELL 06-10C] Built session database: 33,736 sessions
[CELL 06-10C] Saved: C:\anonymous-users-mooc-session-meta\models\baselines\sessionknn.pkl
[CELL 06-10C] elapsed=16.54s
[CELL 06-10C] done


In [15]:
# [CELL 06-10C] Baseline 5: Session-KNN (V-SKNN)

t0 = cell_start("CELL 06-10C", "Session-KNN baseline")

class SessionKNN:
    """
    Session-based k-Nearest Neighbors (V-SKNN variant).

    Finds similar past sessions and recommends items from those sessions.
    Similarity based on cosine similarity of session item sets.
    """
    def __init__(self, n_items: int, k: int = 100, sample_size: int = 500):
        self.n_items = n_items
        self.k = k  # number of nearest neighbor sessions
        self.sample_size = sample_size  # max sessions to consider (for efficiency)
        self.sessions = []  # list of (session_id, item_list)

    def fit(self, pairs_df: pd.DataFrame):
        """Build session database from training data."""
        # Group pairs by user to create sessions
        sessions_list = []
        for user_id, user_pairs in pairs_df.groupby('user_id'):
            # Sort by timestamp
            user_pairs_sorted = user_pairs.sort_values('label_ts_epoch')

            # Extract session as list of items (prefix + label)
            session_items = []
            for _, row in user_pairs_sorted.iterrows():
                prefix = row['prefix']
                label = row['label']
                # Add items from prefix
                if isinstance(prefix, (list, np.ndarray)):
                    session_items.extend(prefix)
                # Add label
                session_items.append(label)

            # Store unique items in session
            session_items_unique = list(dict.fromkeys(session_items))  # preserve order, remove duplicates
            sessions_list.append((user_id, session_items_unique))

        self.sessions = sessions_list
        print(f"[CELL 06-10C] Built session database: {len(self.sessions):,} sessions")

    def _session_similarity(self, session1: List[int], session2: List[int]) -> float:
        """Compute cosine similarity between two sessions (item sets)."""
        set1 = set(session1)
        set2 = set(session2)

        if len(set1) == 0 or len(set2) == 0:
            return 0.0

        # Jaccard similarity (intersection over union) as approximation
        # More efficient than full cosine for sets
        intersection = len(set1 & set2)
        union = len(set1 | set2)

        return intersection / union if union > 0 else 0.0

    def predict(self, current_session: List[int]) -> np.ndarray:
        """
        Predict item scores based on k most similar past sessions.

        Args:
            current_session: List of items in current session (prefix)

        Returns:
            scores: (n_items,) array of prediction scores
        """
        if len(current_session) == 0 or len(self.sessions) == 0:
            return np.zeros(self.n_items)

        # Sample sessions for efficiency if too many
        sessions_to_consider = self.sessions
        if len(self.sessions) > self.sample_size:
            import random
            sessions_to_consider = random.sample(self.sessions, self.sample_size)

        # Compute similarities to all sessions
        similarities = []
        for session_id, session_items in sessions_to_consider:
            sim = self._session_similarity(current_session, session_items)
            similarities.append((session_id, session_items, sim))

        # Sort by similarity (descending)
        similarities.sort(key=lambda x: x[2], reverse=True)

        # Take top-k similar sessions
        top_k_sessions = similarities[:self.k]

        # Aggregate scores from top-k sessions
        scores = np.zeros(self.n_items)
        total_sim = sum(sim for _, _, sim in top_k_sessions)

        if total_sim == 0:
            return scores

        for session_id, session_items, sim in top_k_sessions:
            # Weight by similarity and recency (last items have higher weight)
            for i, item in enumerate(session_items):
                if 0 <= item < self.n_items:
                    # Recency weight: more recent items (later in list) get higher weight
                    recency_weight = (i + 1) / len(session_items)
                    scores[item] += sim * recency_weight

        # Normalize by total similarity
        scores /= total_sim

        return scores

# Train Session-KNN
print("[CELL 06-10C] Training Session-KNN (k=100)...")
sessionknn_model = SessionKNN(n_items=n_items, k=100, sample_size=500)
sessionknn_model.fit(pairs_train)

# Save
save_pickle(MODELS_DIR / 'sessionknn.pkl', sessionknn_model)
print(f"[CELL 06-10C] Saved: {MODELS_DIR / 'sessionknn.pkl'}")

cell_end("CELL 06-10C", t0)


[CELL 06-10C] Session-KNN baseline
[CELL 06-10C] start=2026-01-07T15:22:16
[CELL 06-10C] Training Session-KNN (k=100)...
[CELL 06-10C] Built session database: 33,736 sessions
[CELL 06-10C] Saved: C:\anonymous-users-mooc-session-meta\models\baselines\sessionknn.pkl
[CELL 06-10C] elapsed=16.22s
[CELL 06-10C] done


In [17]:
# [CELL 06-11] Evaluate all baselines on test episodes

t0 = cell_start("CELL 06-11", "Evaluate baselines")

def evaluate_on_episodes(model, episodes_df: pd.DataFrame, pairs_df: pd.DataFrame, model_type: str) -> Dict[str, float]:
    """
    Evaluate a model on episodes.

    For each episode:
    - Use query pairs (ignore support for now, this is zero-shot)
    - Predict next course for each query pair
    - Aggregate metrics
    """
    all_predictions = []
    all_labels = []

    for _, episode in episodes_df.iterrows():
        query_pair_ids = episode["query_pair_ids"]

        # Get query pairs
        query_pairs = pairs_df[pairs_df["pair_id"].isin(query_pair_ids)].sort_values("label_ts_epoch")

        if len(query_pairs) == 0:
            continue

        labels = query_pairs["label"].values

        if model_type == "random":
            preds = model.predict(len(query_pairs))

        elif model_type == "popularity":
            preds = model.predict(len(query_pairs))

        elif model_type == "gru":
            # Use prefix from each query pair
            prefixes = query_pairs["prefix"].tolist()
            preds = []

            model.eval()
            with torch.no_grad():
                for prefix in prefixes:
                    if len(prefix) > CFG["gru_config"]["max_seq_len"]:
                        prefix = prefix[-CFG["gru_config"]["max_seq_len"]:]

                    seq = torch.LongTensor([prefix]).to(DEVICE)
                    length = torch.LongTensor([len(prefix)]).to(DEVICE)
                    logits = model(seq, length)
                    scores = torch.softmax(logits, dim=-1).cpu().numpy()[0]
                    preds.append(scores)

            preds = np.array(preds)

        elif model_type == "sasrec":
            # Use prefix from each query pair
            prefixes = query_pairs["prefix"].tolist()
            preds = []

            model.eval()
            with torch.no_grad():
                for prefix in prefixes:
                    if len(prefix) > 50:  # max_len for SASRec
                        prefix = prefix[-50:]

                    seq = torch.LongTensor([prefix]).to(DEVICE)
                    logits = model(seq)
                    scores = torch.softmax(logits, dim=-1).cpu().numpy()[0]
                    preds.append(scores)

            preds = np.array(preds)

        elif model_type == "sessionknn":
            # Use prefix from each query pair
            prefixes = query_pairs["prefix"].tolist()
            preds = []

            for prefix in prefixes:
                scores = model.predict(prefix)
                preds.append(scores)

            preds = np.array(preds)

        all_predictions.append(preds)
        all_labels.extend(labels)

    # Concatenate all predictions
    all_predictions = np.vstack(all_predictions)
    all_labels = np.array(all_labels)

    # Compute metrics
    metrics = compute_metrics(all_predictions, all_labels, k_values=[5, 10])
    return metrics

# Evaluate on test set
results = {}

print("[CELL 06-11] Evaluating Random...")
results["random"] = evaluate_on_episodes(random_model, episodes_test, pairs_test, "random")
print(f"  Accuracy@1: {results['random']['accuracy@1']:.4f}")

print("\n[CELL 06-11] Evaluating Popularity...")
results["popularity"] = evaluate_on_episodes(popularity_model, episodes_test, pairs_test, "popularity")
print(f"  Accuracy@1: {results['popularity']['accuracy@1']:.4f}")

print("\n[CELL 06-11] Evaluating GRU (global)...")
results["gru_global"] = evaluate_on_episodes(gru_model, episodes_test, pairs_test, "gru")
print(f"  Accuracy@1: {results['gru_global']['accuracy@1']:.4f}")

print("\n[CELL 06-11] Evaluating SASRec...")
results["sasrec"] = evaluate_on_episodes(sasrec_model, episodes_test, pairs_test, "sasrec")
print(f"  Accuracy@1: {results['sasrec']['accuracy@1']:.4f}")

print("\n[CELL 06-11] Evaluating Session-KNN...")
results["sessionknn"] = evaluate_on_episodes(sessionknn_model, episodes_test, pairs_test, "sessionknn")
print(f"  Accuracy@1: {results['sessionknn']['accuracy@1']:.4f}")

# Save results
results_with_meta = {
    "run_id": RUN_ID,
    "k_shot_config": {"K": K, "Q": Q},
    "n_test_episodes": len(episodes_test),
    "baselines": results,
}
write_json_atomic(Path(CFG["outputs"]["results"]), results_with_meta)
print(f"\n[CELL 06-11] Saved: {CFG['outputs']['results']}")

cell_end("CELL 06-11", t0)


[CELL 06-11] Evaluate baselines
[CELL 06-11] start=2026-01-07T15:32:43
[CELL 06-11] Evaluating Random...
  Accuracy@1: 0.0017

[CELL 06-11] Evaluating Popularity...
  Accuracy@1: 0.0260

[CELL 06-11] Evaluating GRU (global)...


C:\Users\User\AppData\Local\Temp\ipykernel_36276\1657306488.py:45: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  seq = torch.LongTensor([prefix]).to(DEVICE)


  Accuracy@1: 0.3373

[CELL 06-11] Evaluating SASRec...
  Accuracy@1: 0.1919

[CELL 06-11] Evaluating Session-KNN...
  Accuracy@1: 0.1500

[CELL 06-11] Saved: C:\anonymous-users-mooc-session-meta\results\baselines_K5_Q10.json
[CELL 06-11] elapsed=20.27s
[CELL 06-11] done


In [18]:
# [CELL 06-12] Results summary table

t0 = cell_start("CELL 06-12", "Results summary")

print("\n[CELL 06-12] ========== BASELINE RESULTS (Test Set) ==========")
print(f"K={K}, Q={Q} | Test Episodes: {len(episodes_test):,}\n")

# Table header
print(f"{'Model':<20} {'Acc@1':>10} {'Recall@5':>10} {'Recall@10':>10} {'MRR':>10}")
print("-" * 62)

for model_name, metrics in results.items():
    print(f"{model_name:<20} {metrics['accuracy@1']:>10.4f} {metrics['recall@5']:>10.4f} {metrics['recall@10']:>10.4f} {metrics['mrr']:>10.4f}")

cell_end("CELL 06-12", t0)


[CELL 06-12] Results summary
[CELL 06-12] start=2026-01-07T15:33:55

[CELL 06-12] ========== BASELINE RESULTS (Test Set) ==========
K=5, Q=10 | Test Episodes: 346

Model                     Acc@1   Recall@5  Recall@10        MRR
--------------------------------------------------------------
random                   0.0017     0.0104     0.0243     0.0164
popularity               0.0260     0.1182     0.2145     0.0892
gru_global               0.3373     0.5590     0.6575     0.4437
sasrec                   0.1919     0.4806     0.6023     0.3318
sessionknn               0.1500     0.4653     0.5775     0.2967
[CELL 06-12] elapsed=0.00s
[CELL 06-12] done


In [19]:
# [CELL 06-10C] Baseline 5: Session-KNN (V-SKNN)

t0 = cell_start("CELL 06-10C", "Session-KNN baseline")

class SessionKNN:
    """
    Session-based k-Nearest Neighbors (V-SKNN variant).

    Finds similar past sessions and recommends items from those sessions.
    Similarity based on cosine similarity of session item sets.
    """
    def __init__(self, n_items: int, k: int = 100, sample_size: int = 500):
        self.n_items = n_items
        self.k = k  # number of nearest neighbor sessions
        self.sample_size = sample_size  # max sessions to consider (for efficiency)
        self.sessions = []  # list of (session_id, item_list)

    def fit(self, pairs_df: pd.DataFrame):
        """Build session database from training data."""
        # Group pairs by user to create sessions
        sessions_list = []
        for user_id, user_pairs in pairs_df.groupby('user_id'):
            # Sort by timestamp
            user_pairs_sorted = user_pairs.sort_values('label_ts_epoch')

            # Extract session as list of items (prefix + label)
            session_items = []
            for _, row in user_pairs_sorted.iterrows():
                prefix = row['prefix']
                label = row['label']
                # Add items from prefix
                if isinstance(prefix, (list, np.ndarray)):
                    session_items.extend(prefix)
                # Add label
                session_items.append(label)

            # Store unique items in session
            session_items_unique = list(dict.fromkeys(session_items))  # preserve order, remove duplicates
            sessions_list.append((user_id, session_items_unique))

        self.sessions = sessions_list
        print(f"[CELL 06-10C] Built session database: {len(self.sessions):,} sessions")

    def _session_similarity(self, session1: List[int], session2: List[int]) -> float:
        """Compute cosine similarity between two sessions (item sets)."""
        set1 = set(session1)
        set2 = set(session2)

        if len(set1) == 0 or len(set2) == 0:
            return 0.0

        # Jaccard similarity (intersection over union) as approximation
        # More efficient than full cosine for sets
        intersection = len(set1 & set2)
        union = len(set1 | set2)

        return intersection / union if union > 0 else 0.0

    def predict(self, current_session: List[int]) -> np.ndarray:
        """
        Predict item scores based on k most similar past sessions.

        Args:
            current_session: List of items in current session (prefix)

        Returns:
            scores: (n_items,) array of prediction scores
        """
        if len(current_session) == 0 or len(self.sessions) == 0:
            return np.zeros(self.n_items)

        # Sample sessions for efficiency if too many
        sessions_to_consider = self.sessions
        if len(self.sessions) > self.sample_size:
            import random
            sessions_to_consider = random.sample(self.sessions, self.sample_size)

        # Compute similarities to all sessions
        similarities = []
        for session_id, session_items in sessions_to_consider:
            sim = self._session_similarity(current_session, session_items)
            similarities.append((session_id, session_items, sim))

        # Sort by similarity (descending)
        similarities.sort(key=lambda x: x[2], reverse=True)

        # Take top-k similar sessions
        top_k_sessions = similarities[:self.k]

        # Aggregate scores from top-k sessions
        scores = np.zeros(self.n_items)
        total_sim = sum(sim for _, _, sim in top_k_sessions)

        if total_sim == 0:
            return scores

        for session_id, session_items, sim in top_k_sessions:
            # Weight by similarity and recency (last items have higher weight)
            for i, item in enumerate(session_items):
                if 0 <= item < self.n_items:
                    # Recency weight: more recent items (later in list) get higher weight
                    recency_weight = (i + 1) / len(session_items)
                    scores[item] += sim * recency_weight

        # Normalize by total similarity
        scores /= total_sim

        return scores

# Train Session-KNN
print("[CELL 06-10C] Training Session-KNN (k=100)...")
sessionknn_model = SessionKNN(n_items=n_items, k=100, sample_size=500)
sessionknn_model.fit(pairs_train)

# Save
save_pickle(MODELS_DIR / 'sessionknn.pkl', sessionknn_model)
print(f"[CELL 06-10C] Saved: {MODELS_DIR / 'sessionknn.pkl'}")

cell_end("CELL 06-10C", t0)


[CELL 06-10C] Session-KNN baseline
[CELL 06-10C] start=2026-01-07T15:34:00
[CELL 06-10C] Training Session-KNN (k=100)...
[CELL 06-10C] Built session database: 33,736 sessions
[CELL 06-10C] Saved: C:\anonymous-users-mooc-session-meta\models\baselines\sessionknn.pkl
[CELL 06-10C] elapsed=16.45s
[CELL 06-10C] done


## ✅ Notebook 06 Complete

**Outputs:**
- ✅ Baseline models: `random.pkl`, `popularity.pkl`, `gru_global.pth`
- ✅ Evaluation results: `results/baselines_K5_Q10.json`
- ✅ `reports/06_baselines_xuetangx/<run_tag>/report.json`

**Baseline Performance (Test Set):**
- Random: Sanity check (1/n_items accuracy)
- Popularity: Recommends most popular courses
- GRU (Global): Trained on all training pairs, zero-shot on test users

**Next Steps:**
1. **Meta-learning**: Implement MAML/Prototypical Networks for few-shot adaptation
2. **Support set**: Use K support pairs from each episode for adaptation
3. **Compare**: Meta-learning vs baselines on same cold-start test users